In [ ]:
\list

In [ ]:
SELECT keywords_all FROM public_job_listings.data_nerd_jobs LIMIT 100;

## Extracting Keywords from JSON
Extracting the entire `keywords_all` column and processing it in Python due to SQL extraction challenges.

In [ ]:
SELECT keywords_all
FROM public_job_listings.data_nerd_jobs
LIMIT 100;

In [ ]:
SELECT list.element AS unpacked_keywords
FROM  public_job_listings.data_nerd_jobs, UNNEST (keywords_all.list) AS  list
LIMIT 5;

In [ ]:
SELECT list.element AS skill, COUNT(*) AS frequency
FROM public_job_listings.data_nerd_jobs, UNNEST(keywords_all.list) AS list
GROUP BY list.element
ORDER BY frequency DESC
LIMIT 10;

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data from the previous SQL query
skill_data = sql_df_jtsj

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='frequency', y='skill', data=skill_data, palette='Blues_r')
plt.title('Top 10 Skills in Job Listings')
plt.xlabel('Frequency')
plt.ylabel('Skill')
plt.show()